# DATA SCIENCE PROJECT 
BY CHARGE EDU-TECH 

DONE BY MANISHA SHARMA 

PROJECT 1 ---- FAKE NEWS DETECTION USING PYTHON

Fake news do not require any introduction. 
It is very much easy to spread all the fake information in today’s all-connected 
world across the internet. Fake news is sometimes transmitted through the 
internet by some unauthorised sources, which creates issues for the targeted 
person and it makes them panic and leads to even violence. 
To combat the spread of fake news, it’s critical to determine the information’s 
legitimacy, which this Data Science project can help with. To do so, Python can 
be used, and a model is created using TfidfVectorizer. Passive Aggressive
Classifier can be implemented to distinguish between true and fake news. 
Pandas, NumPy, and sci-kit-learn are some Python packages suitable for this 
project, and we can utilize News.csv for the dataset.
    


# Fake News Detection Model using TensorFlow in Python

using fake_news_dataset, which contains News text and corresponding label (FAKE or REAL).

Dataset can be downloaded from this link : using fake_news_dataset, which contains News text and corresponding label (FAKE or REAL). 

Dataset can be downloaded from this link ('news.csv')


# The steps to be followed are : 

1. Importing Libraries and dataset
2. Preprocessing Dataset
3. Generating Word Embeddings
4. Model Architecture
5. Model Evaluation and Prediction

# Importing Libraries and Dataset

In [1]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()


In [3]:
import numpy as np
import pandas as pd

# Reading the data
data = pd.read_csv("C:/Users/Lenovo/Downloads/news (1).csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# Preprocessing Dataset

we can see the dataset contains one unnamed column. So we drop that column from the dataset.

In [4]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# Data Encoding

It converts the categorical column (label in out case) into numerical values.

In [5]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

These are some variables required for the model training.

In [6]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

# Tokenization 

This process divides a large piece of continuous text into distinct units or tokens basically. Here we use columns separately for a temporal basis as a pipeline just for good accuracy.

In [7]:
title = []
text = []
labels = []
for x in range(training_size):
	title.append(data['title'][x])
	text.append(data['text'][x])
	labels.append(data['label'][x])

Applying Tokenization

In [8]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

# Generating Word Embedding

It allows words with similar meanings to have a similar representation. 
Here each individual word is represented as real-valued vectors in a predefined vector space.
For that we will use glove.6B.50d.txt. 
It has the predefined vector space for words. 
You can download the file using this link.

In [9]:
embeddings_index = {}
with open('C://Users//Lenovo//Downloads//glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1 + 1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


# Creating Model Architecture

Now it’s time to introduce TensorFlow to create the model. 
Here we use the TensorFlow embedding technique with Keras Embedding Layer where we map original input data into some set of real-valued dimensions.

In [10]:
model = tf.keras.Sequential([
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
							input_length=max_length, weights=[
								embeddings_matrix],
							trainable=False),
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Conv1D(64, 5, activation='relu'),
	tf.keras.layers.MaxPooling1D(pool_size=4),
	tf.keras.layers.LSTM(64),
	tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
			optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [11]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
					epochs=num_epochs,
					validation_data=(testing_padded,
									testing_labels),
					verbose=2)


Train on 2700 samples, validate on 300 samples
Epoch 1/50


C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6379 - accuracy: 0.6219 - val_loss: 0.5610 - val_accuracy: 0.7033 - 3s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 2s - loss: 0.5803 - accuracy: 0.6989 - val_loss: 0.5722 - val_accuracy: 0.7000 - 2s/epoch - 735us/sample
Epoch 3/50
2700/2700 - 2s - loss: 0.5464 - accuracy: 0.7178 - val_loss: 0.5124 - val_accuracy: 0.7067 - 2s/epoch - 677us/sample
Epoch 4/50
2700/2700 - 2s - loss: 0.5025 - accuracy: 0.7515 - val_loss: 0.6038 - val_accuracy: 0.6867 - 2s/epoch - 705us/sample
Epoch 5/50
2700/2700 - 2s - loss: 0.4548 - accuracy: 0.7856 - val_loss: 0.5173 - val_accuracy: 0.7033 - 2s/epoch - 701us/sample
Epoch 6/50
2700/2700 - 2s - loss: 0.3808 - accuracy: 0.8344 - val_loss: 0.4860 - val_accuracy: 0.7667 - 2s/epoch - 695us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3428 - accuracy: 0.8456 - val_loss: 0.4992 - val_accuracy: 0.7500 - 2s/epoch - 710us/sample
Epoch 8/50
2700/2700 - 2s - loss: 0.3237 - accuracy: 0.8537 - val_loss: 0.5067 - val_accuracy: 0.7600 - 2s/epoch - 68

# Model Evaluation and Prediction

Now, the detection model is built using TensorFlow. Now we will try to test the model by using some news text by predicting whether it is true or false.

In [12]:
# sample text to check if fake or not
X = "Karry to go to France in gesture of sympathy"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
						padding=padding_type,
						truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
	print("This news is True")
else:
	print("This news is false")


C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


This news is false
